**ROLL NO: 21i-2682**

**SECTION: DS-N**

**AI - ASSIGNMENT 1**





**DFS APPROACH**

In [1]:
#utalizing the source code provided
def solve_cryptarithmetic_dfs():
    def is_valid_assignment(assignment):
        #check if leading digits are not zero
        if assignment['S'] == 0 or assignment['M'] == 0:
            return False
        return True

    def dfs(letters, digits, assigned, assignment):
        if len(assigned) == len(letters):
            if is_valid_assignment(assignment):
                send = 1000 * assignment['S'] + 100 * assignment['E'] + 10 * assignment['N'] + assignment['D']
                more = 1000 * assignment['M'] + 100 * assignment['O'] + 10 * assignment['R'] + assignment['E']
                money = 10000 * assignment['M'] + 1000 * assignment['O'] + 100 * assignment['N'] + 10 * assignment['E'] + assignment['Y']
                if send + more == money:
                    return assignment
            return None

        for digit in digits:
            if digit not in assigned.values():
                letter = letters[len(assigned)]
                assigned[letter] = digit
                assignment[letter] = digit
                result = dfs(letters, digits, assigned, assignment)
                if result:
                    return result
                del assigned[letter]
                del assignment[letter]

        return None

    letters = ('S', 'E', 'N', 'D', 'M', 'O', 'R', 'Y')
    digits = range(10)
    assigned = {}
    assignment = {}
    return dfs(letters, digits, assigned, assignment)

print(solve_cryptarithmetic_dfs())


{'S': 9, 'E': 5, 'N': 6, 'D': 7, 'M': 1, 'O': 0, 'R': 8, 'Y': 2}


**UCS**

In [12]:
import heapq
import time

def timeit(fn):
    def wrapper():
        start = time.perf_counter()
        ret = fn()
        elapsed = time.perf_counter() - start
        print("%s took %.2fs" % (fn.__name__, elapsed))
        return ret
    return wrapper

class State:
    def __init__(self, assignment, cost):
        self.assignment = assignment
        self.cost = cost

    def __lt__(self, other):
        return self.cost < other.cost

def solve_cryptarithmetic_ucs():
    def is_valid_assignment(assignment):
        #check if leading digits are not zero
        if assignment['S'] == 0 or assignment['M'] == 0:
            return False
        return True

    letters = ('S', 'E', 'N', 'D', 'M', 'O', 'R', 'Y')

    #initialize the priority queue with the initial state
    priority_queue = [State({}, 0)]
    while priority_queue:
        #pop the state with the lowest cost
        state = heapq.heappop(priority_queue)
        assignment = state.assignment
        if len(assignment) == len(letters):
            #check if the assignment is valid
            if is_valid_assignment(assignment):
                send = 1000 * assignment['S'] + 100 * assignment['E'] + 10 * assignment['N'] + assignment['D']
                more = 1000 * assignment['M'] + 100 * assignment['O'] + 10 * assignment['R'] + assignment['E']
                money = 10000 * assignment['M'] + 1000 * assignment['O'] + 100 * assignment['N'] + 10 * assignment['E'] + assignment['Y']
                if send + more == money:
                    return assignment
        else:
            #expand the current state by adding one more letter
            for digit in range(10):
                if digit not in assignment.values():
                    new_assignment = assignment.copy()
                    new_assignment[letters[len(assignment)]] = digit
                    #calculate the cost (total number of assignments)
                    new_cost = sum(1 for _ in new_assignment.values())
                    heapq.heappush(priority_queue, State(new_assignment, new_cost))
    return None

@timeit
def solve():
    return solve_cryptarithmetic_ucs()

print(solve())

solve took 26.23s
{'S': 9, 'E': 5, 'N': 6, 'D': 7, 'M': 1, 'O': 0, 'R': 8, 'Y': 2}


**DFS**

In [7]:
import re
import time

from z3 import *

def cryptarithmetic_dfs(input: str, limit=None, unique=True):
    start_time  = time.perf_counter()
    solver      = Solver()
    token_words = re.findall(r'\b[a-zA-Z]\w*\b', input)  #words must start with a letter

    letters = { l: Int(l) for l in list("".join(token_words)) }
    words   = { w: Int(w) for w in list(token_words)          }

    #convert letters to numbers
    for l,s in letters.items(): solver.add(0 <= s, s <= 9)

    #letters must be unique
    if unique and len(letters) <= 10:
        solver.add(Distinct(*letters.values()))

    #words must be unique
    solver.add(Distinct(*words.values()))

    #first letter must not be zero
    for word in words.keys():
        solver.add( letters[word[0]] != 0 )

    #convert words to decimal values
    for word, word_symbol in words.items():
        solver.add(word_symbol == Sum(*[
            letter_symbol * 10**index
            for index,letter_symbol in enumerate(reversed([
                letters[l] for l in list(word)
                ]))
            ]))

    #problem def as defined by input
    solver.add(eval(input, None, words))

    solutions = []
    stack = [(solver, [])]  # to keep track of solver state and assignments
    while stack:
        current_solver, current_assignment = stack.pop()
        if str(current_solver.check()) == 'sat':
            model = current_solver.model()
            solution = {str(s): model[s] for w, s in words.items()}
            solutions.append(solution)
            #to avoid finding the same solution again
            stack.append((current_solver, current_assignment + [Or([s != model[s] for _, s in words.items()])]))
            #for deeper search
            stack.append((current_solver, current_assignment + [s != model[s] for _, s in words.items()]))
        if limit and len(solutions) >= limit:
            break

    run_time = round(time.perf_counter() - start_time, 1)
    print(f'{len(solutions)} solutions found in {run_time}s')
    return solutions

#'TWO + TWO == FOUR
#('SEND + MORE == MONEY')
#ONE * TWO == THREE
#HERE + SHE == COMES
input_string = "XY - X == YX"
solutions = cryptarithmetic_dfs(input_string, limit=1)
if solutions:
    print("Solution:", solutions[0])
else:
    print("No solution found")


1 solutions found in 0.0s
Solution: {'XY': 98, 'X': 9, 'YX': 89}


In [2]:
pip install z3-solver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 16.4 MB/s eta 0:00:00


**GREEDY APPROACH**

In [11]:
import re
import time
from collections import Counter
from z3 import *

def heuristic(letters, words):
    #fefine a heuristic function to estimate the priority of assignments
    letter_counts = Counter("".join(words.keys()))
    return lambda l: -letter_counts[l]

def cryptarithmetic_greedy(input: str, limit=None, unique=True):
    start_time  = time.perf_counter()
    solver      = Solver()
    token_words = re.findall(r'\b[a-zA-Z]\w*\b', input)  #words start with a letter

    letters = { l: Int(l) for l in list("".join(token_words)) }
    words   = { w: Int(w) for w in list(token_words)          }

    #convert letters to numbers
    for l, s in letters.items(): solver.add(0 <= s, s <= 9)

    #letters must be unique
    if unique and len(letters) <= 10:
        solver.add(Distinct(*letters.values()))

    #words must be unique
    solver.add(Distinct(*words.values()))

    #first letter must not be zero
    for word in words.keys():
        solver.add( letters[word[0]] != 0 )

    #convert words to decimal values
    for word, word_symbol in words.items():
        solver.add(word_symbol == Sum(*[
            letter_symbol * 10**index
            for index, letter_symbol in enumerate(reversed([
                letters[l] for l in list(word)
            ]))
        ]))

    #Problem definition as defined by input
    solver.add(eval(input, None, words))

    solutions = []
    priority_queue = [(heuristic(letters, words), solver, [])]  #Priority queue to keep track of the solver state and assignments
    while priority_queue:
        _, current_solver, current_assignment = priority_queue.pop(0)  #POP the state with the highest priority
        if str(current_solver.check()) == 'sat':
            model = current_solver.model()
            solution = {str(s): model[s] for w, s in words.items()}
            solutions.append(solution)
            #ADd constraints to avoid finding same solution again
            current_solver_copy = current_solver
            current_solver_copy.add(Or([s != model[s] for _, s in words.items()]))
            priority_queue.append((heuristic(letters, words), current_solver_copy, current_assignment))
            #deeper search
            current_solver_deep = current_solver
            current_solver_deep.add([s != model[s] for _, s in words.items()])
            priority_queue.append((heuristic(letters, words), current_solver_deep, current_assignment))
        if limit and len(solutions) >= limit:
            break

        #Sort priority queue based on priority function
        priority_queue.sort(key=lambda x: x[0](x[2][-1]) if x[2] else float('-inf'))

    run_time = round(time.perf_counter() - start_time, 1)
    print(f'{len(solutions)} solutions found in {run_time}s')
    return solutions

input_string = "SEND + MORE == MONEY"
solutions = cryptarithmetic_greedy(input_string, limit=1)
if solutions:
    print("Solution:", solutions[0])
else:
    print("No solution found")


1 solutions found in 0.0s
Solution: {'SEND': 9567, 'MORE': 1085, 'MONEY': 10652}


In [4]:
import re
import time
from collections import Counter
from z3 import *

def heuristic(letters, words):
    #define a heuristic function to estimate the priority of assignments
    letter_counts = Counter("".join(words.keys()))
    return lambda l: -letter_counts[l]

def a_star_search(input: str, limit=None, unique=True):
    start_time  = time.perf_counter()
    solver      = Solver()
    token_words = re.findall(r'\b[a-zA-Z]\w*\b', input)  # Words start with a letter

    letters = { l: Int(l) for l in list("".join(token_words)) }
    words   = { w: Int(w) for w in list(token_words)          }

    #convert letters to num
    for l, s in letters.items():
        solver.add(0 <= s, s <= 9)

    #letters must be unique
    if unique and len(letters) <= 10:
        solver.add(Distinct(*letters.values()))

    #words must be unique
    solver.add(Distinct(*words.values()))

    #1st letter must not be zero
    for word in words.keys():
        solver.add(letters[word[0]] != 0)

    #convert words to decimal values
    for word, word_symbol in words.items():
        solver.add(word_symbol == Sum(*[
            letter_symbol * 10**index
            for index, letter_symbol in enumerate(reversed([
                letters[l] for l in list(word)
            ]))
        ]))

    solver.add(eval(input, None, words))

    solutions = []
    priority_queue = [(0, solver, [])]  #queue to keep know solver state and assignments
    while priority_queue:
        _, current_solver, current_assignment = priority_queue.pop(0)  #pop the state with the highest priority
        if str(current_solver.check()) == 'sat':
            model = current_solver.model()
            solution = {str(s): model[s] for w, s in words.items()}
            solutions.append(solution)
            if limit and len(solutions) >= limit:
                break

            #new states
            for w, s in words.items():
                new_solver = current_solver.copy()
                new_solver.add(s != model[s])  #finding the same solution again-avoid it
                priority_queue.append((len(current_assignment) + 1 + heuristic(letters, words)(w), new_solver, current_assignment + [s]))

            #sort priority queue based on priority function
            priority_queue.sort(key=lambda x: x[0])

    run_time = round(time.perf_counter() - start_time, 1)
    print(f'{len(solutions)} solutions found in {run_time}s')
    return solutions

input_string = "XY - X == YX"
solutions = a_star_search(input_string, limit=1)
if solutions:
    print("Solution:", solutions[0])
else:
    print("No solution found")


1 solutions found in 0.1s
Solution: {'XY': 98, 'X': 9, 'YX': 89}
